# Get data for ABSA
-----

**Data list**  
  
\< FULL \>
1. SemEval-2016 task5 Subtask1 (another notebook)
2. SemEval-2016 task5 Subtask2 (here)

**Task Description**  

    Aspect Based Sentiment Analysis by full reviews(documents) (not sentences)

In [3]:
import os, sys
import xml.etree.ElementTree as ET

DIR = 'data_raw'
path_rest = os.path.join(DIR, 'ABSA16_Restaurants_Train_English_SB2.xml')
path_laptop = os.path.join(DIR, 'ABSA16_Laptops_Train_English_SB2.xml')

root_rest = ET.parse(path_rest).getroot()
root_laptop = ET.parse(path_laptop).getroot()

In [6]:
# Data structure
f = open(path_rest, 'r')
f.readlines()

['<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n',
 '<Reviews>\n',
 '    <Review rid="1004293">\n',
 '        <sentences>\n',
 '            <sentence id="1004293:0">\n',
 '                <text>Judging from previous posts this used to be a good place, but not any longer.</text>\n',
 '            </sentence>\n',
 '            <sentence id="1004293:1">\n',
 '                <text>We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.</text>\n',
 '            </sentence>\n',
 '            <sentence id="1004293:2">\n',
 '                <text>They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.</text>\n',
 '            </sentence>\n',
 '            <sentence id="1004293:3">\n',
 '                <text>The food was lousy - too sweet or too salty and the portions tiny.</text>\n',
 '            </sentence>\n',
 '            <sentence

In [37]:
import pandas as pd

class semeval16_sub2:
    
    def __init__(self, path): # same features in both datasets
        self.root = ET.parse(path).getroot()
        print('Element Tree parsed')
        print('Process:\n1)reviews_list\n2)make_dict\n3)export_csv')
        
    def reviews_list(self):
        'get sentences(ET parsed) list'
        self.reviews = self.root.findall('Review')
        
    def make_dict(self):
        'return dictionary all conatains'
        
        self.re_idx, self.idx, self.review, self.category, self.polarity = \
            list(), list(), list(), list(), list()
        self.dict_ = dict()
        
        for index, contents in enumerate(self.reviews):
            if len(contents) != 2:
                continue
            
            raw_idx = contents.attrib['rid']
            text = str()
            sentences = contents[0].findall('sentence')
            opinions = contents[1].findall('Opinion')
            num_opinions = len(opinions)
            
            self.re_idx += [index] * num_opinions
            self.idx += [raw_idx] * num_opinions
            
            # full review sentences
            for s in sentences:
                temp = s[0].text + ' '
                text += temp
            self.review += [text[:-1]] * num_opinions
            
            # opinions
            for o in opinions:
                self.category.append(o.attrib['category'])
                self.polarity.append(o.attrib['polarity'])
        
        self.dict_['re_idx'], self.dict_['idx'], self.dict_['review'], self.dict_['category'], self.dict_['polarity'] = \
            self.re_idx, self.idx, self.review, self.category, self.polarity
        
        return self.dict_
    
    def export_csv(self, export_path):
        df = pd.DataFrame(self.dict_)
        df.to_csv(export_path, index=False)
        print('dataset has been exported')

In [42]:
laptop = semeval16_sub2(path=path_laptop)
laptop.reviews_list()
laptop_dict = laptop.make_dict()
laptop.export_csv('semeval16_sub2_laptop.csv')

Element Tree parsed
Process:
1)reviews_list
2)make_dict
3)export_csv
dataset has been exported


In [43]:
rest = semeval16_sub2(path=path_rest)
rest.reviews_list()
rest_dict = rest.make_dict()
rest.export_csv('semeval16_sub2_restaurant.csv')

Element Tree parsed
Process:
1)reviews_list
2)make_dict
3)export_csv
dataset has been exported


In [46]:
pd.DataFrame(rest_dict)[:9]

,re_idx,idx,review,category,polarity
0,0,1004293,Judging from previous posts this used to be a ...,RESTAURANT#GENERAL,negative
1,0,1004293,Judging from previous posts this used to be a ...,SERVICE#GENERAL,negative
2,0,1004293,Judging from previous posts this used to be a ...,FOOD#QUALITY,negative
3,0,1004293,Judging from previous posts this used to be a ...,FOOD#STYLE_OPTIONS,negative
4,1,1014458,"I have eaten at Saul, many times, the food is ...",FOOD#QUALITY,positive
5,1,1014458,"I have eaten at Saul, many times, the food is ...",RESTAURANT#GENERAL,positive
6,1,1014458,"I have eaten at Saul, many times, the food is ...",DRINKS#STYLE_OPTIONS,positive
7,1,1014458,"I have eaten at Saul, many times, the food is ...",DRINKS#PRICES,positive
8,1,1014458,"I have eaten at Saul, many times, the food is ...",RESTAURANT#PRICES,positive
